# Part 1 — Modèle de détection de mail/message frauduleux (Phishing)

Objectif:
- Entraîner un modèle de classification (phishing vs safe) sur des données textuelles
- Évaluer les performances (Precision/Recall/F1, etc.)
- Sauvegarder les artifacts (vectorizer + model)
- Générer `ref_data.csv` (données de référence vectorisées) pour la suite (monitoring / drift / retrain)


In [ ]:
# Imports & configuration
import os
import re
import json
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score
)

RANDOM_SEED = 42

# Chemins des données
VALIDATION_DATA_PATH = "../data/Phishing_validation_emails.csv"
PROD_DATA_PATH = "../data/prod_data.csv"

# Output path:
ARTIFACT_DIR = "../artifacts"
DATA_OUT_DIR = "../data"
os.makedirs(ARTIFACT_DIR, exist_ok=True)
os.makedirs(DATA_OUT_DIR, exist_ok=True)

print("OK: dirs ready")

OK: dirs ready


In [ ]:
# Lecture des données de validation
df_validation = pd.read_csv(VALIDATION_DATA_PATH)
print("📊 Données de validation:", df_validation.shape)

# Lecture des données de production (feedbacks utilisateurs)
if os.path.exists(PROD_DATA_PATH):
    df_prod = pd.read_csv(PROD_DATA_PATH, header=None, names=["Email Text", "model_prediction", "Email Type"])
    # On garde seulement les colonnes nécessaires
    df_prod = df_prod[["Email Text", "Email Type"]]
    print("📊 Données de production:", df_prod.shape)
    
    # Concaténation des deux sources
    df = pd.concat([df_validation, df_prod], ignore_index=True)
    print("✅ Données combinées:", df.shape)
else:
    print("⚠️  Pas de prod_data.csv, utilisation uniquement des données de validation")
    df = df_validation

display(df.head())

print("\nColumns:", df.columns.tolist())
print("\nMissing values:\n", df.isna().sum())

# Distribution d'étiquettes
print("\nLabel distribution:\n", df["Email Type"].value_counts())

(2000, 2)


,Email Text,Email Type
0,"Dear Jordan, your subscription has been succes...",Safe Email
1,"Dear Casey, thank you for your purchase. Your ...",Safe Email
2,Congratulations! You've won a $3000 gift card....,Phishing Email
3,You have a new secure message from your bank. ...,Phishing Email
4,Your package delivery is pending. Please provi...,Phishing Email



Columns: ['Email Text', 'Email Type']

Missing values:
 Email Text    0
Email Type    0
dtype: int64

Label distribution:
 Email Type
Safe Email        1000
Phishing Email    1000
Name: count, dtype: int64


## Présentation du jeu de données – Détection de phishing

Le jeu de données utilisé dans ce projet est destiné à la **détection de mails/messages frauduleux (phishing)** à partir de données textuelles.

Il contient **2000 emails** répartis en deux classes équilibrées :
- **Safe Email** : 1000 messages légitimes
- **Phishing Email** : 1000 messages frauduleux

### Structure des données
Le fichier CSV comporte **2 colonnes** :
- **Email Text** : contenu textuel de l’email
- **Email Type** : étiquette associée au message (`Safe Email` ou `Phishing Email`)

### Qualité des données
- Aucune valeur manquante n’est présente dans le jeu de données
- Les classes sont parfaitement équilibrées, ce qui facilite l’entraînement et l’évaluation des modèles de classification
- Les textes sont courts à moyens, représentatifs de messages réels (notifications, alertes bancaires, confirmations, etc.)

### Utilisation dans le projet
Ce jeu de données est utilisé comme :
- **jeu de données d’entraînement et de validation** pour le modèle de détection de phishing
- **jeu de données de référence (`ref_data`)** pour la phase de monitoring, de détection de dérive et de ré-entraînement automatique du modèle dans une approche MLOps

Il constitue une base adaptée pour un **prototype fonctionnel** et une démonstration complète du cycle de vie d’un modèle de détection de fraude basé sur des données textuelles.


In [9]:
# Nettoyage de base (normalisation du texte + mappage des labels)
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip().lower()
    # Fusionner les espaces multiples
    s = re.sub(r"\s+", " ", s)
    return s

# Uniformisation des noms de colonnes (optionnel)
df = df.rename(columns={"Email Text": "text", "Email Type": "label_str"})

df["text"] = df["text"].apply(normalize_text)

# Mappage des labels : Phishing=1, Safe=0
label_map = {
    "Phishing Email": 1,
    "Safe Email": 0
}
df["label"] = df["label_str"].map(label_map)

# Vérifier s’il y a des labels non mappés
bad = df[df["label"].isna()]
print("Unmapped labels rows:", len(bad))
if len(bad) > 0:
    display(bad.head())

df["label"] = df["label"].astype(int)

display(df.head())


Unmapped labels rows: 0


,text,label_str,label
0,"dear jordan, your subscription has been succes...",Safe Email,0
1,"dear casey, thank you for your purchase. your ...",Safe Email,0
2,congratulations! you've won a $3000 gift card....,Phishing Email,1
3,you have a new secure message from your bank. ...,Phishing Email,1
4,your package delivery is pending. please provi...,Phishing Email,1


## Séparation des données en Train / Validation / Test (70% / 15% / 15%)
- **Train** : utilisé pour entraîner le modèle  

- **Validation** : utilisé pour l’ajustement des hyperparamètres, la sélection du modèle et l’arrêt anticipé (*early stopping*)  

- **Test** : utilisé uniquement à la toute fin pour évaluer la capacité réelle de généralisation du modèle  


In [10]:
# Séparation des données en Train / Validation / Test (70% / 15% / 15%)

X = df["text"].values      # Textes des emails (features)
y = df["label"].values     # Labels numériques (0 = safe, 1 = phishing)

# Première séparation : 70% entraînement, 30% temporaire (validation + test)
X_train, X_tmp, y_train, y_tmp = train_test_split(
    X,
    y,
    test_size=0.30,              # 30% pour validation + test
    random_state=RANDOM_SEED, 
    stratify=y                   # Conserver la proportion des classes
)

# Deuxième séparation : 15% validation, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp,
    y_tmp,
    test_size=0.50,              # Diviser le 30% en deux parties égales
    random_state=RANDOM_SEED,
    stratify=y_tmp               # Même distribution des labels
)

# Affichage des tailles des ensembles
print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))


Train: 1400 Val: 300 Test: 300


In [12]:
# Construction d’un modèle baseline (TF-IDF + Régression Logistique)
# Modèle léger, rapide et interprétable — idéal pour un PoC et le déploiement

model = Pipeline(steps=[
    (
        "tfidf",
        TfidfVectorizer(
            ngram_range=(1, 2),     # unigrammes + bigrammes
            min_df=2,               # ignorer les mots trop rares
            max_features=30000      # limiter la dimension du vocabulaire
        )
    ),
    (
        "clf",
        LogisticRegression(
            max_iter=1000,          # assurer la convergence
            class_weight="balanced",# gérer le déséquilibre des classes
            random_state=RANDOM_SEED
        )
    )
])

# Renvoyer l'objet modèle pour vérification structurelle
model


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=30000, min_df=2,
                                 ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    random_state=42))])

In [13]:
# Entraîner le modèle
model.fit(X_train, y_train)
print("Training done")

Training done


## Évaluation du modèle (réutilisable)

Cette étape définit une fonction d’évaluation générique pour les modèles de classification binaire.
Elle permet de mesurer de manière cohérente les performances du modèle sur différents jeux de données
(validation et test).

Les métriques calculées incluent :
- Accuracy
- Precision
- Recall
- F1-score
- Matrice de confusion
- Rapport de classification détaillé

In [15]:
def eval_model(pipeline, X_eval, y_eval, name="Eval"):
    # Prédiction des classes (0 / 1)
    y_pred = pipeline.predict(X_eval)

    # Prédiction des probabilités
    # LogisticRegression supporte predict_proba
    # [:, 1] correspond à la probabilité de la classe positive (phishing)
    y_proba = pipeline.predict_proba(X_eval)[:, 1]

    # Calcul de l'accuracy
    acc = accuracy_score(y_eval, y_pred)

    # Calcul des métriques Precision, Recall et F1 (classification binaire)
    p, r, f1, _ = precision_recall_fscore_support(
        y_eval, y_pred, average="binary"
    )

    # Affichage des résultats principaux
    print(f"== {name} dataset ==")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {p:.4f}")
    print(f"Recall   : {r:.4f}")
    print(f"F1       : {f1:.4f}")

    # Affichage de la matrice de confusion
    print("\nConfusion matrix:\n", confusion_matrix(y_eval, y_pred))

    # Rapport de classification détaillé par classe
    print(
        "\nClassification report:\n",
        classification_report(
            y_eval,
            y_pred,
            target_names=["Safe", "Phishing"]
        )
    )

    # Retourner les métriques sous forme de dictionnaire
    return {
        "accuracy": acc,
        "precision": p,
        "recall": r,
        "f1": f1
    }


# Évaluation sur le jeu de validation
val_metrics = eval_model(model, X_val, y_val, name="Validation")

# Évaluation finale sur le jeu de test
test_metrics = eval_model(model, X_test, y_test, name="Test")


== Validation dataset ==
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1       : 1.0000

Confusion matrix:
 [[150   0]
 [  0 150]]

Classification report:
               precision    recall  f1-score   support

        Safe       1.00      1.00      1.00       150
    Phishing       1.00      1.00      1.00       150

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

== Test dataset ==
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1       : 1.0000

Confusion matrix:
 [[150   0]
 [  0 150]]

Classification report:
               precision    recall  f1-score   support

        Safe       1.00      1.00      1.00       150
    Phishing       1.00      1.00      1.00       150

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



### Commentaire sur les résultats

Les performances parfaites (Accuracy, Precision, Recall et F1 = 1.00) obtenues sur les jeux de validation et de test
s’expliquent principalement par la nature du jeu de données.

Le dataset est fortement nettoyé, parfaitement équilibré (50 % Safe / 50 % Phishing) et contient des signaux lexicaux
très discriminants entre les deux classes. Dans ce contexte, un modèle linéaire simple (TF-IDF + Régression Logistique)
est capable de séparer les classes sans difficulté.

Ces résultats ne reflètent pas des conditions réelles et ne doivent pas être interprétés comme une capacité de
généralisation en environnement de production.


### Interprétation des caractéristiques phishing

Cette analyse permet d’identifier les termes (mots ou n-grammes) qui contribuent le plus
à la prédiction de la classe *Phishing* dans le modèle de régression logistique.

Dans un modèle linéaire, chaque coefficient positif élevé indique que la présence du terme
augmente fortement la probabilité qu’un email soit classé comme phishing.

Les 20 termes affichés correspondent donc aux signaux lexicaux les plus discriminants
appris par le modèle à partir des données d’entraînement.

Cette étape est utile pour :
- comprendre le comportement du modèle,
- détecter une dépendance excessive à certains mots-clés,
- concevoir des jeux de données plus difficiles (hard negatives),
- évaluer la robustesse du modèle face à des emails plus réalistes.

In [17]:
# Extraire les poids appris par la régression logistique
# Chaque poids correspond à un terme TF-IDF
coef = model.named_steps["clf"].coef_[0]

# Récupérer la liste des termes (vocabulaire) du TF-IDF
features = model.named_steps["tfidf"].get_feature_names_out()

# Associer chaque terme à son poids,
# puis trier par poids décroissant (signal phishing le plus fort)
top_phishing = sorted(
    zip(features, coef),
    key=lambda x: x[1],
    reverse=True
)[:20]

# Afficher les 20 caractéristiques les plus indicatives du phishing
top_phishing


[('to', 1.9926915830924015),
 ('account', 1.686574980637964),
 ('click', 1.5529384413831087),
 ('your account', 1.4113542154229286),
 ('your', 1.338783834106305),
 ('delivery', 1.223645869806031),
 ('avoid', 1.2205620004642428),
 ('to avoid', 1.2205620004642428),
 ('click here', 1.1668373097220364),
 ('here to', 1.1668373097220364),
 ('continue', 1.1562592952286272),
 ('to continue', 1.1562592952286272),
 ('in', 1.118494661177496),
 ('service', 1.0144449544845138),
 ('update your', 1.0144449544845138),
 ('information', 1.0102045074541943),
 ('information to', 1.0102045074541943),
 ('unusual', 1.0002553735908055),
 ('here', 0.8916220942586611),
 ('and pay', 0.8475448831656011)]

In [23]:
top_safe = sorted(
    zip(features, coef),
    key=lambda x: x[1]
)[:20]

# Afficher les 20 caractéristiques les plus indicatives du safe mails
top_safe

[('for', -2.0274924043330356),
 ('the', -1.435096490995501),
 ('hi', -1.2016612148936046),
 ('meeting', -1.1351166002949915),
 ('for your', -1.1254714682838023),
 ('thank', -1.1254714682838023),
 ('thank you', -1.1254714682838023),
 ('you for', -1.1254714682838023),
 ('let', -1.1031741300580564),
 ('on', -1.0176959668636572),
 ('project', -1.0176959668636572),
 ('the project', -1.0176959668636572),
 ('find', -0.9963069384496878),
 ('please find', -0.9963069384496878),
 ('be', -0.9535627443505413),
 ('updates', -0.9372341315913835),
 ('dear', -0.8930742386230505),
 ('you', -0.8644879636854478),
 ('reminder', -0.8606623960966574),
 ('at', -0.8535312576857301)]

In [20]:
hard_samples = [
    # Contenu de phishing, mais avec suppression volontaire des mots-clés forts
    "Please review the attached document regarding your recent request.",
    "We noticed an unusual activity related to your profile. More details inside.",
    
    # Contenu légitime (safe), mais avec ajout de mots-clés typiques du phishing
    "Thank you for your order, please click here to see the invoice.",
    "Your account information has been updated successfully."
]

# Prédiction des classes (0 = safe, 1 = phishing)
hard_pred = model.predict(hard_samples)

# Prédiction des probabilités associées à la classe phishing
hard_proba = model.predict_proba(hard_samples)[:, 1]

# Affichage des résultats pour chaque email
for t, y, p in zip(hard_samples, hard_pred, hard_proba):
    print("-" * 60)
    print(t)
    print(
        "Pred:",
        "phishing" if y == 1 else "safe",
        "| proba:",
        round(float(p), 3)
    )


------------------------------------------------------------
Please review the attached document regarding your recent request.
Pred: safe | proba: 0.36
------------------------------------------------------------
We noticed an unusual activity related to your profile. More details inside.
Pred: phishing | proba: 0.827
------------------------------------------------------------
Thank you for your order, please click here to see the invoice.
Pred: safe | proba: 0.305
------------------------------------------------------------
Your account information has been updated successfully.
Pred: phishing | proba: 0.827


### Analyse des hard samples
**Ce résultat a permis de valider avec succès l'efficacité du modèle.**  
Les résultats obtenus sur les *hard samples* montrent que le modèle ne se base pas uniquement sur la présence explicite de mots-clés, mais reste néanmoins fortement influencé par certains signaux lexicaux.

L’analyse des coefficients du modèle montre que la détection repose
principalement sur des mots-clés spécifiques (*click*, *account*, *update*, etc.).

Cette dépendance rend le modèle vulnérable à des emails frauduleux
qui évitent volontairement ces termes, ou à des emails légitimes
contenant des formulations similaires.

Des exemples artificiellement construits montrent que le modèle
peut être trompé, ce qui justifie la mise en place d’un monitoring
et d’un ré-entraînement continu en production.

#### Analyse des caractéristiques et lien avec la suite du projet

L’analyse des poids du modèle de régression logistique montre que la détection
du phishing repose principalement sur des **mots-clés et expressions spécifiques**
tels que *click*, *account*, *update*, ou *click here*.  
Ces termes constituent des signaux forts permettant d’obtenir d’excellentes
performances sur le jeu de données étudié.

Cependant, cette dépendance à des **caractéristiques lexicales de surface**
rend le modèle potentiellement vulnérable à l’évolution des stratégies de phishing,
où les attaquants peuvent éviter ou modifier ces mots-clés.

**Donc** c’est pourquoi, dans une logique de mise en production, le projet intègre
un **mécanisme de retour utilisateur et de ré-entraînement automatique du modèle
via un agent IA**, afin de s’adapter progressivement à de nouvelles formes de messages
frauduleux et de réduire la dépendance à des motifs fixes.


In [24]:
# Sauvegarde des artefacts (modèle + métriques)
# Comme nous utilisons un Pipeline, il est plus simple de sauvegarder l’ensemble du pipeline
# Pipeline = TF-IDF + Logistic Regression
artifact_path = os.path.join(ARTIFACT_DIR, "phishing_tfidf_logreg.joblib")
joblib.dump(model, artifact_path)
print("Saved:", artifact_path)

# Sauvegarde des métriques d’évaluation au format JSON pour le suivi des expériences
metrics_path = os.path.join(ARTIFACT_DIR, "metrics.json")
with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(
        {"val": val_metrics, "test": test_metrics},
        f,
        ensure_ascii=False,
        indent=2
    )

print("Saved:", metrics_path)


Saved: ./artifacts\phishing_tfidf_logreg.joblib
Saved: ./artifacts\metrics.json


### Génération des données de référence (ref_data.csv)

Cette étape consiste à transformer les textes du jeu de données en vecteurs TF-IDF à l’aide du
vectoriseur entraîné, puis à convertir ces représentations en un tableau de caractéristiques.

Le fichier `ref_data.csv` obtenu contient :
- les variables explicatives issues du TF-IDF,
- la variable cible (`target`).

Ces données servent de **référence** pour le monitoring du modèle après déploiement, notamment
pour la détection de *data drift* et de *model drift*.  

In [25]:
# Extraction du composant TF-IDF depuis le pipeline
tfidf: TfidfVectorizer = model.named_steps["tfidf"]

# Utilisation de l'ensemble du dataset comme données de référence
# (il est aussi possible d'utiliser uniquement le jeu d'entraînement)
X_ref_text = df["text"].values
y_ref = df["label"].values

# Transformation des textes en vecteurs TF-IDF (matrice creuse)
X_ref_vec = tfidf.transform(X_ref_text)
feature_names = tfidf.get_feature_names_out()

# Conversion en DataFrame sparse
# Attention : le DataFrame peut être large et volumineux
ref_df = pd.DataFrame.sparse.from_spmatrix(
    X_ref_vec,
    columns=feature_names
)

# Ajout de la variable cible (label)
ref_df["target"] = y_ref

# Sauvegarde des données de référence pour le monitoring / drift
ref_path = os.path.join(DATA_OUT_DIR, "ref_data.csv")
ref_df.to_csv(ref_path, index=False)

print("Saved ref_data:", ref_path)
print("ref_data shape:", ref_df.shape)


Saved ref_data: ./data\ref_data.csv
ref_data shape: (2000, 501)


In [26]:
# Validation rapide : charger le modèle et prédire un échantillon
loaded = joblib.load(os.path.join(ARTIFACT_DIR, "phishing_tfidf_logreg.joblib"))

sample_texts = [
    "Dear user, your account will be suspended, click here to verify immediately.",
    "Bonjour, voici le compte rendu de la réunion de demain. Merci."
]

pred = loaded.predict(sample_texts)
proba = loaded.predict_proba(sample_texts)[:, 1]

for t, yhat, p in zip(sample_texts, pred, proba):
    print("-" * 60)
    print("TEXT:", t)
    print("PRED:", "phishing(1)" if yhat == 1 else "safe(0)")
    print("PROBA(phishing):", round(float(p), 4))


------------------------------------------------------------
TEXT: Dear user, your account will be suspended, click here to verify immediately.
PRED: phishing(1)
PROBA(phishing): 0.8569
------------------------------------------------------------
TEXT: Bonjour, voici le compte rendu de la réunion de demain. Merci.
PRED: safe(0)
PROBA(phishing): 0.4543


## Synthèse – Partie 1 : Détection de mails/messages frauduleux

Dans cette première partie, nous avons développé un **modèle de détection de phishing basé sur des données textuelles**.  
Le jeu de données contient 2000 emails équilibrés entre messages légitimes et frauduleux, sans valeurs manquantes.

Un modèle **TF-IDF + régression logistique** a été entraîné après une phase de nettoyage et de normalisation des textes.  
Les performances obtenues sur les jeux de validation et de test sont très élevées, ce qui s’explique par la présence de **signaux lexicaux forts** (mots-clés typiques du phishing).

Une analyse des coefficients du modèle montre que la décision repose principalement sur ces expressions, ce qui met en évidence la **nécessité d’un monitoring et d’un ré-entraînement continu** en contexte réel.  
Les artifacts du modèle ainsi qu’un jeu de données de référence (`ref_data`) ont été générés pour servir de base aux étapes suivantes (API, monitoring, agent de ré-entraînement).
